In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import f1_score,accuracy_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
import multiprocessing

In [3]:
data_train=pd.read_excel('C:/Users/Administrator/Desktop/Datathon/datasets-v5/task 1/train.xlsx')
data_test=pd.read_excel('C:/Users/Administrator/Desktop/Datathon/datasets-v5/test data/test-INPUT/task-1/task1.test.xlsx').dropna()
# print(data_train.head())
data_test.head()

,news_text,id,type_find
0,Chicago police are seeking the publicâ€™s help...,100013.0,?
1,Moment of silence at Palestine Pavilion during...,100015.0,?
2,The Organization of Islamic Cooperation (OIC) ...,100024.0,?
3,The chairman of the House Oversight and Govern...,100031.0,?
4,A judge in the United Kingdom has sentenced a ...,100040.0,?


In [9]:
import pandas_profiling as pf
pf.ProfileReport(data_train)

Number of variables,3
Number of observations,35993
Total Missing (%),0.1%
Total size in memory,843.7 KiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [10]:
import pandas_profiling as pf
pf.ProfileReport(data_test)

Number of variables,4
Number of observations,10150
Total Missing (%),0.0%
Total size in memory,317.3 KiB
Average record size in memory,32.0 B
Numeric,1
Categorical,0
Boolean,0
Date,0
Text (Unique),1
Rejected,2


In [12]:
data_test.isnull().sum()

news_text    0
id           0
type_find    0
dtype: int64

In [5]:
import os
path='C:/Users/Administrator/'
os.chdir(path)
print(os.getcwd())

C:\Users\Administrator


In [6]:
data_train['news_text']=data_train['news_text'].str.lower().str.replace('[^0-9a-z ]','')
data_test['news_text']=data_test['news_text'].str.lower().str.replace('[^0-9a-z ]','')
data_test['news_text'][0]

'chicago police are seeking the publics help in locating a 61yearold man last seen tuesday on the south side  willie woods was last seen around 5 pm in the 1200 block of east 72nd street in the grand crossing neighborhood police said in a missing persons alert  he is black with brown eyes and black hair is 5foot7 and 140 pounds  woods reportedly carries identification  anyone with information is asked to call area central detectives at 3127478380'

In [7]:
#docs=' '.join(data['news_text'].str)
stemmer=PorterStemmer()
stopwords=nltk.corpus.stopwords.words('english')
def clean_data(text):
    words=nltk.word_tokenize(text)
    words=[stemmer.stem(word) for word in words if word not in stopwords]
    return " ".join(words)

In [8]:
new_clean_data_train=pd.DataFrame()
for i in range(data_train.shape[0]):
    try:
        new_data_train={'news_text':str(clean_data(data_train.iloc[i]['news_text'])),
                        'id':data_train.iloc[i]['id'],
                        'type':data_train.iloc[i]['type']}
    except:continue
    new_clean_data_train=new_clean_data_train.append(new_data_train,ignore_index=True)

In [9]:
new_clean_data_train=new_clean_data_train.dropna()
new_clean_data_train.head()

,id,news_text,type
0,727600136.0,et tu rhodi recent editori provid journal cata...,non-propaganda
1,731714618.0,recent post farmington mirror town version oni...,non-propaganda
2,731714635.0,presid donald trump often respond natur disast...,non-propaganda
3,728627182.0,februari black histori month noth loom larger ...,non-propaganda
4,728627443.0,snow heavi whip gust wind travel nearli imposs...,non-propaganda


In [10]:
new_clean_data_test=pd.DataFrame()
for i in range(data_test.shape[0]):
    try:
        new_data_test={'news_text':str(clean_data(data_test.iloc[i]['news_text'])),
                       'id':data_test.iloc[i]['id']}
    except:continue
    new_clean_data_test=new_clean_data_test.append(new_data_test,ignore_index=True)

In [12]:
new_clean_data_test=new_clean_data_test.dropna()
new_clean_data_test.head()

,news_text
id,
100013.0,chicago polic seek public help locat 61yearold...
100015.0,moment silenc palestin pavilion cann film fest...
100024.0,organ islam cooper oic hold extraordinari summ...
100031.0,chairman hous oversight govern reform committe...
100040.0,judg unit kingdom sentenc scientist univers ac...


In [20]:
# x=new_clean_data_train.drop('type',axis=1)
# y=new_clean_data_test['type']
# train,test=train_test_split(x['news_text'],y,test_size=0.2,random_state=100)
train_x=new_clean_data_train['news_text']
train_y=new_clean_data_train['type']
test_x=new_clean_data_test['news_text']
test_y=new_clean_data_train['type']

In [ ]:
# vectorizer=CountVectorizer()
# cv_train_x=vectorizer.fit_transform(train_x)
# cv_test_x=vectorizer.transform(test_x)

# vectorizer=TfidfVectorizer()
# tf_train_x=vectorizer.fit_transform(train_x)
# tf_test_x=vectorizer.transform(test_x)

# vectorizer=HashingVectorizer()
# h_train_x=vectorizer.fit_transform(train_x)
# h_test_x=vectorizer.transform(test_x)

# dt_model=DecisionTreeClassifier()
# dt_model.fit(cv_train_x,train_y)
# pred=dt_model.predict(cv_test_x)
# print("accuracy is: %0.3f"%accuracy_score(pred,test_y))
# print('f1_score is: %0.3f'%f1_score(pred,test_y, average='weighted'))

In [14]:
def vector_variable(vector,x1,x2):
    vec=vector()
    return vec.fit_transform(x1), vec.transform(x2)

In [15]:
def get_score(model,x1,y1,x2,y2):
    import warnings
    warnings.filterwarnings("ignore")
    model_name=model
    model=model()
    model.fit(x1,y1)
    pred=model.predict(x2)
    (print('f1_score for ',model_name,' is: %0.3f'%f1_score(pred,y2, average='weighted')))  

In [24]:
tf_train,tf_test=vector_variable(TfidfVectorizer,train_x,test_x)

In [21]:
# tf_train_x,tf_test_x=vector_variable(TfidfVectorizer,train_x,test_x)

In [23]:
get_score(LogisticRegression,tf_train_x,train_y,tf_test_x,test_y)
get_score(MultinomialNB,tf_train_x,train_y,tf_test_x,test_y)
get_score(PassiveAggressiveClassifier,tf_train_x,train_y,tf_test_x,test_y)
get_score(DecisionTreeClassifier,tf_train_x,train_y,tf_test_x,test_y)
get_score(RandomForestClassifier,tf_train_x,train_y,tf_test_x,test_y)
get_score(AdaBoostClassifier,tf_train_x,train_y,tf_test_x,test_y)
get_score(XGBClassifier,tf_train_x,train_y,tf_test_x,test_y)
get_score(SVC,tf_train_x,train_y,tf_test_x,test_y)

ValueError: Found input variables with inconsistent numbers of samples: [10150, 35956]

In [19]:
cv_train_x,cv_test_x=vector_variable(CountVectorizer,train_x,test_x)

In [20]:
get_score(LogisticRegression,cv_train_x,train_y,cv_test_x,test_y)
get_score(MultinomialNB,cv_train_x,train_y,cv_test_x,test_y)
get_score(PassiveAggressiveClassifier,cv_train_x,train_y,cv_test_x,test_y)
get_score(DecisionTreeClassifier,cv_train_x,train_y,cv_test_x,test_y)
get_score(RandomForestClassifier,cv_train_x,train_y,cv_test_x,test_y)
get_score(AdaBoostClassifier,cv_train_x,train_y,cv_test_x,test_y)
get_score(XGBClassifier,cv_train_x,train_y,cv_test_x,test_y)
get_score(SVC,cv_train_x,train_y,cv_test_x,test_y)

f1_score for  <class 'sklearn.linear_model.logistic.LogisticRegression'>  is: 0.954
f1_score for  <class 'sklearn.naive_bayes.MultinomialNB'>  is: 0.925
f1_score for  <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'>  is: 0.946
f1_score for  <class 'sklearn.tree.tree.DecisionTreeClassifier'>  is: 0.928
f1_score for  <class 'sklearn.ensemble.forest.RandomForestClassifier'>  is: 0.937
f1_score for  <class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>  is: 0.943
f1_score for  <class 'xgboost.sklearn.XGBClassifier'>  is: 0.949
f1_score for  <class 'sklearn.svm.classes.SVC'>  is: 0.941


In [25]:
pa_model=PassiveAggressiveClassifier()
pa_model.fit(tf_train,train_y)
pred=pa_model.predict(tf_test)
update_article=pd.DataFrame({'Articles':test_x.index,'Predicted_type':pred})

In [26]:
#update_article.to_csv('Predicted Result.csv',index=False)
update_article.head()

,Articles,Predicted_type
0,100013.0,non-propaganda
1,100015.0,non-propaganda
2,100024.0,non-propaganda
3,100031.0,non-propaganda
4,100040.0,non-propaganda
